<br><font size=6><b>Efficient <font color=#383838><img style="display: inline;margin-left:-12px;" src="Data/yelp-logo.png" width="14%"/>API</font></b></font><font size=6> Calls</font>



<font size=3><ul><li>Suganthan Thamotharampillai
<br><li>01.31.2023

<div style="border-style:solid;border-color:red;border-width:10px;background-image: url("https://static.vecteezy.com/system/resources/thumbnails/008/490/121/small/wave-abstract-lines-background-line-art-png.png");">
asfasfasfsdf<br>
a<br>
a<br>
aa<br>
a<br>
aa<br>
a<br>
aa<br>
a<br>
aa<br>
a<br>
aa<br>
a<br>
a
</div>

In [ ]:
one